In [ ]:
import json
from py2neo import Graph, Node, Relationship

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(n_nodes, n_relationships))

In [ ]:
import time

query = """
MATCH (q:Quanta)
WHERE 
    q.year >= 1960 AND
    (q.venue= "The American Economic Review" OR 
    q.venue= "Econometrica" OR
    q.venue = "Journal of Political Economy" OR
    q.venue = "Quarterly Journal of Economics" OR
    q.venue = "The Review of Economic Studies")
RETURN 
    q.venue as journal, 
    avg(q.pageRank_2018) as impact
"""

query_start_time = time.time()
df = graph.run(query).to_data_frame()
print("Finished ({:.2f} minutes).".format((time.time()-query_start_time)/60))    
end_time = time.time()
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = df.sort_values(ascending=False,by="impact")

sns.set(style="ticks", palette="muted")
ax = sns.barplot(y="journal", x="impact", data=df);

impact_factors = [3.379, 4.73,5.247,5.920,4.038]
bar_labels = ["IF: {:.2f}".format(i) for i in impact_factors]
pos = range(5)
for tick,label in zip(pos,ax.get_xticklabels()):
    ax.text(1, range(5)[tick]+.1, bar_labels[tick], 
            horizontalalignment='center', size='x-small', color='w', weight='semibold')

ax.set(xlabel='Average Article PageRank from 1950-2017 Citation Network', ylabel='Journal')
plt.show()